## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np

print('Libraries loaded')

## 2. Load Individual Model Forecasts

In [ ]:
# Load forecasts from all three models
lstm_forecast = pd.read_csv('lstm_ha15m_forecast.csv')
rf_forecast = pd.read_csv('randomforest_ha15m_forecast.csv')
xgb_forecast = pd.read_csv('xgboost_ha15m_forecast.csv')

print(f'LSTM shape: {lstm_forecast.shape}')
print(f'Random Forest shape: {rf_forecast.shape}')
print(f'XGBoost shape: {xgb_forecast.shape}')

print(f'\nLSTM columns: {lstm_forecast.columns.tolist()}')
print(f'RF columns: {rf_forecast.columns.tolist()}')
print(f'XGB columns: {xgb_forecast.columns.tolist()}')

## 3. Merge Forecasts

In [ ]:
# Merge all forecasts
ensemble_df = pd.DataFrame()
ensemble_df['Time'] = lstm_forecast['Time']
ensemble_df['LSTM_Pred'] = lstm_forecast.iloc[:, 1]  # Second column
ensemble_df['RF_Pred'] = rf_forecast.iloc[:, 1]
ensemble_df['XGB_Pred'] = xgb_forecast.iloc[:, 1]

print(f'Ensemble dataframe shape: {ensemble_df.shape}')
print(f'\nFirst 10 rows:')
print(ensemble_df.head(10))

## 4. Majority Voting

In [ ]:
# Ensemble voting logic:
# 2+ models predict 1 → signal = 1 (BULLISH)
# 2+ models predict -1 → signal = -1 (BEARISH)
# Split decision (1 votes each) → signal = 0 (NEUTRAL)

def ensemble_vote(lstm, rf, xgb):
    votes = [lstm, rf, xgb]
    bullish = sum(1 for v in votes if v == 1)
    bearish = sum(1 for v in votes if v == -1)
    
    if bullish >= 2:
        return 1  # Consensus BULLISH
    elif bearish >= 2:
        return -1  # Consensus BEARISH
    else:
        return 0  # No consensus

ensemble_df['Ensemble_Vote'] = ensemble_df.apply(
    lambda row: ensemble_vote(row['LSTM_Pred'], row['RF_Pred'], row['XGB_Pred']),
    axis=1
)

print('Ensemble voting complete!')
print(f'\nVote distribution:')
print(ensemble_df['Ensemble_Vote'].value_counts())

## 5. Calculate Agreement Metrics

In [ ]:
# Count how many models agree
def count_agreement(lstm, rf, xgb):
    votes = [lstm, rf, xgb]
    agreement_count = 0
    
    # Count matching predictions
    if lstm == rf:
        agreement_count += 1
    if lstm == xgb:
        agreement_count += 1
    if rf == xgb:
        agreement_count += 1
    
    return agreement_count  # 0 to 3

ensemble_df['Agreement_Count'] = ensemble_df.apply(
    lambda row: count_agreement(row['LSTM_Pred'], row['RF_Pred'], row['XGB_Pred']),
    axis=1
)

# Agreement percentage
ensemble_df['Consensus_Strength'] = (ensemble_df['Agreement_Count'] / 3) * 100

print('Agreement metrics calculated')
print(f'\nAgreement distribution:')
print(ensemble_df['Agreement_Count'].value_counts().sort_index())

## 6. Summary Statistics

In [ ]:
bullish_signals = (ensemble_df['Ensemble_Vote'] == 1).sum()
bearish_signals = (ensemble_df['Ensemble_Vote'] == -1).sum()
neutral_signals = (ensemble_df['Ensemble_Vote'] == 0).sum()

print(f'Ensemble Voting Results:')
print(f'  Bullish signals: {bullish_signals} ({bullish_signals/len(ensemble_df)*100:.1f}%)')
print(f'  Bearish signals: {bearish_signals} ({bearish_signals/len(ensemble_df)*100:.1f}%)')
print(f'  Neutral signals: {neutral_signals} ({neutral_signals/len(ensemble_df)*100:.1f}%)')

avg_consensus = ensemble_df['Consensus_Strength'].mean()
print(f'\nAverage Consensus Strength: {avg_consensus:.1f}%')
print(f'  Strong consensus (100%): {(ensemble_df[\"Consensus_Strength\"] == 100).sum()}')
print(f'  Moderate consensus (67%): {(ensemble_df[\"Consensus_Strength\"] == 66.67).sum()}')
print(f'  Weak consensus (33%): {(ensemble_df[\"Consensus_Strength\"] == 33.33).sum()}')

## 7. Confidence Score

In [ ]:
# Confidence = percentage of models agreeing with majority
def calculate_confidence(lstm, rf, xgb, vote):
    if vote == 0:  # Neutral
        return 33.33  # Lowest confidence
    
    agreeing = 0
    if lstm == vote:
        agreeing += 1
    if rf == vote:
        agreeing += 1
    if xgb == vote:
        agreeing += 1
    
    return (agreeing / 3) * 100

ensemble_df['Confidence'] = ensemble_df.apply(
    lambda row: calculate_confidence(
        row['LSTM_Pred'], row['RF_Pred'], row['XGB_Pred'], row['Ensemble_Vote']
    ),
    axis=1
)

print('Confidence scores calculated')
print(f'Average confidence: {ensemble_df[\"Confidence\"].mean():.2f}%')

## 8. Save Ensemble Forecast

In [ ]:
# Create output CSV with essential columns
output_df = ensemble_df[['Time', 'LSTM_Pred', 'RF_Pred', 'XGB_Pred', 'Ensemble_Vote', 'Confidence']].copy()

# Rename for clarity
output_df.columns = ['Time', 'LSTM', 'RandomForest', 'XGBoost', 'Ensemble', 'Confidence']

output_df.to_csv('ensemble_ha15m_forecast.csv', index=False)

print('Ensemble forecast saved: ensemble_ha15m_forecast.csv')
print(f'Shape: {output_df.shape}')
print(f'\nFirst 10 rows:')
print(output_df.head(10))

## 9. Model Agreement Analysis

In [ ]:
# Check pairwise agreement between models
lstm_rf_agree = (ensemble_df['LSTM_Pred'] == ensemble_df['RF_Pred']).sum()
lstm_xgb_agree = (ensemble_df['LSTM_Pred'] == ensemble_df['XGB_Pred']).sum()
rf_xgb_agree = (ensemble_df['RF_Pred'] == ensemble_df['XGB_Pred']).sum()

total = len(ensemble_df)

print('Pairwise Agreement:')
print(f'  LSTM ↔ RF:   {lstm_rf_agree}/{total} ({lstm_rf_agree/total*100:.1f}%)')
print(f'  LSTM ↔ XGB:  {lstm_xgb_agree}/{total} ({lstm_xgb_agree/total*100:.1f}%)')
print(f'  RF ↔ XGB:    {rf_xgb_agree}/{total} ({rf_xgb_agree/total*100:.1f}%)')

## 10. Final Summary

In [ ]:
print('='*60)
print('ENSEMBLE VOTING SUMMARY')
print('='*60)
print(f'\nTotal Bars: {len(ensemble_df)}')
print(f'\nSignal Distribution:')
print(f'  Bullish (1):   {bullish_signals:6d} ({bullish_signals/total*100:5.1f}%)')
print(f'  Bearish (-1):  {bearish_signals:6d} ({bearish_signals/total*100:5.1f}%)')
print(f'  Neutral (0):   {neutral_signals:6d} ({neutral_signals/total*100:5.1f}%)')
print(f'\nConsensus Metrics:')
print(f'  Average Consensus: {avg_consensus:.1f}%')
print(f'  Average Confidence: {ensemble_df[\"Confidence\"].mean():.1f}%')
print(f'\nModel Agreement:')
print(f'  LSTM ↔ RF:   {lstm_rf_agree/total*100:.1f}%')
print(f'  LSTM ↔ XGB:  {lstm_xgb_agree/total*100:.1f}%')
print(f'  RF ↔ XGB:    {rf_xgb_agree/total*100:.1f}%')
print(f'\nOutput File: ensemble_ha15m_forecast.csv')
print('='*60)